# Proyecto Final - Precios de SUVs 

In [1]:
%load_ext autoreload
%autoreload 2

### Analisis dataset crudo

In [1]:
import pandas as pd
# Cargar el dataset
df_raw = pd.read_csv("dataset/raw/pf_suvs_i302_1s2025.csv")
df_raw = pd.DataFrame(df_raw)

In [3]:
from src.data_exploration import*

In [4]:
exploratory_analysis(df_raw, target_col="Precio")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18254 entries, 0 to 18253
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               18254 non-null  int64  
 1   Marca                    18254 non-null  object 
 2   Modelo                   18254 non-null  object 
 3   Año                      18254 non-null  float64
 4   Versión                  18254 non-null  object 
 5   Color                    17865 non-null  object 
 6   Tipo de combustible      18254 non-null  object 
 7   Puertas                  18254 non-null  float64
 8   Transmisión              18239 non-null  object 
 9   Motor                    18216 non-null  object 
 10  Tipo de carrocería       18254 non-null  object 
 11  Con cámara de retroceso  4691 non-null   object 
 12  Kilómetros               18254 non-null  object 
 13  Título                   18254 non-null  object 
 14  Precio                

In [5]:
df_raw["Moneda"].value_counts()

Moneda
$      11158
US$     7096
Name: count, dtype: int64

In [6]:
df_raw["Marca"].value_counts()

Marca
Ford              2161
Jeep              2050
Volkswagen        2037
Chevrolet         1750
Renault           1491
Toyota            1260
Peugeot           1250
Nissan            1059
Citroën            721
BMW                672
Honda              597
Hyundai            568
Audi               529
Fiat               366
Chery              344
Kia                290
Mercedes-Benz      282
Dodge              185
BAIC               165
Suzuki              76
Porsche             67
Land Rover          65
Mitsubishi          47
Volvo               38
SsangYong           30
D.S.                27
Alfa Romeo          25
JAC                 19
Jetour              14
GWM                  9
Haval                9
Isuzu                8
Lifan                7
Lexus                7
Subaru               5
Daihatsu             4
D·S                  4
Mini                 3
Hiunday              2
KAIYI                2
DS AUTOMOBILES       2
Jaguar               2
Range Rover          1
Rrena

In [7]:
df_raw["Transmisión"].value_counts()

Transmisión
Automática               12281
Manual                    5834
Automática secuencial      123
Semiautomática               1
Name: count, dtype: int64

In [8]:
df_raw["Tipo de vendedor"].value_counts()

Tipo de vendedor
concesionaria    8509
particular       5676
tienda           4069
Name: count, dtype: int64

In [9]:
df_raw["Tipo de carrocería"].value_counts()

Tipo de carrocería
SUV    18254
Name: count, dtype: int64

In [10]:
df_raw["Tipo de combustible"].value_counts()

Tipo de combustible
Nafta            16067
Diésel            1328
Nafta/GNC          451
Híbrido/Nafta      269
Híbrido            105
GNC                 12
Eléctrico           12
Mild Hybrid         10
Name: count, dtype: int64

In [2]:
from src.data_cleaner import DataProcessor

### Pre-procesamientos del dataset

In [12]:
# usar distintos tipos de datasets
dp_OH = DataProcessor(df_raw)
df_OH = pd.DataFrame(dp_OH.preprocess())
df_OH.to_csv("dataset/processed/marcas_OH.csv", index=False)

dp_OH_no_grouping = DataProcessor(df_raw, config={
    "group_transmission": False,
    "group_combustible": False})
df_OH_no_grouping = pd.DataFrame(dp_OH_no_grouping.preprocess())
df_OH_no_grouping.to_csv("dataset/processed/marcas_OH_no_grouping.csv", index=False)

df_no_brands = df_raw.copy()
df_no_brands = df_no_brands.drop(columns=["Marca", "Modelo"], errors="ignore")  
dp_no_brands_grouping = DataProcessor(df_no_brands, config={
    "one_hot_encode": False
})
df_no_brands_grouping = dp_no_brands_grouping.preprocess()
df_no_brands_grouping.to_csv("dataset/processed/sin_marcas_grouping.csv", index=False)


dp_no_brands_no_grouping = DataProcessor(df_no_brands, config={
    "one_hot_encode": False,
    "group_transmission": False,
    "group_combustible": False})
df_no_brands_no_grouping = pd.DataFrame(dp_no_brands_no_grouping.preprocess())
df_no_brands_no_grouping.to_csv("dataset/processed/sin_marcas_no_grouping.csv", index=False)

In [13]:
exploratory_analysis(df_no_brands_grouping, target_col="Precio_usd")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17781 entries, 0 to 17780
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Puertas              17781 non-null  float64
 1   Kilómetros           17781 non-null  float64
 2   Precio_usd           17781 non-null  float64
 3   Antigüedad           17781 non-null  float64
 4   Cilindrada           17781 non-null  float64
 5   Transmisión_Manual   17781 non-null  int64  
 6   vendedor_particular  17781 non-null  int64  
 7   combustible_Diésel   17781 non-null  bool   
 8   combustible_Nafta    17781 non-null  bool   
 9   combustible_Otros    17781 non-null  bool   
dtypes: bool(3), float64(5), int64(2)
memory usage: 1.0 MB
None

--- Resumen Estadístico ---
            Puertas    Kilómetros     Precio_usd     Antigüedad    Cilindrada  \
count  17781.000000  1.778100e+04   17781.000000   17781.000000  17781.000000   
mean      18.522243  7.518157e+05   

In [14]:
from models.linear_regression import LinearReg
from src.train_val_models import*

### Comparar metricas de varios modelos

In [15]:
datasets = {
    "One-hot completo": (dp_OH, df_OH)
}

resultados = run_experiment(datasets)


[One-hot completo] MSE: 354291393.07 | RMSE: 18822.63 | MAE: 11624.33


Probe crear una columna gama pero usando el precio, creo que hay data leak

In [27]:
from src.cross_val import cross_validate_rf, tune_and_test_rf

### Random forest parameters exploration

In [ ]:

import pandas as pd
from sklearn.ensemble import RandomForestRegressor


datasets_explorations = [
    ("One-hot completo", df_OH),
    ("One-hot sin agrupamiento", df_OH_no_grouping),
    ("Sin marcas con agrupamiento", df_no_brands_grouping),
    ("Sin marcas sin agrupamiento", df_no_brands_no_grouping)
]

param_grid = {
    "n_trees": [20, 50,100,200],             
    "max_depth": [5, 20, 50,None],         
    "min_samples_split": [2, 5, 10],        
    "max_features": [5,20, "sqrt", "log2"] 
}
resultados = []

for nombre, dataset in datasets_explorations:
    print(f"\nEvaluando dataset: {nombre}")
    res = tune_and_test_rf(dataset, target_col="Precio_usd", param_grid=param_grid, top_n=3)
    print(res)
    resultados.append({
    "dataset": nombre,
    "best_params_val": res["best_params_val"],
    "val_rmse": res["val_rmse"],
    "best_params_test": res["best_params_test"],
    "test_rmse": res["test_rmse"]
})






📊 Evaluando dataset: One-hot completo
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 2, 'max_features': 5} --> RMSE promedio: 16565.2956
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 2, 'max_features': 20} --> RMSE promedio: 13058.9772
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 2, 'max_features': 'sqrt'} --> RMSE promedio: 13976.8248
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 2, 'max_features': 'log2'} --> RMSE promedio: 15733.0827
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 5, 'max_features': 5} --> RMSE promedio: 16576.6969
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 5, 'max_features': 20} --> RMSE promedio: 13211.9844
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 5, 'max_features': 'sqrt'} --> RMSE promedio: 14095.3699
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 5, 'max_features': 'log2'} --> RMSE promedio: 15847.7209
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 10, 'max_features': 5} --> RMSE promedio: 1659

In [ ]:
df_resultados = pd.DataFrame(resultados)

for _, row in df_resultados.iterrows():
    print(f"\n📌 Dataset: {row['dataset']}")
    print(f"🔧 Best Val Params:  {row['best_params_val']}")
    print(f"📉 Val RMSE:         {row['val_rmse']:.2f}")
    print(f"🏆 Best Test Params: {row['best_params_test']}")
    print(f"🧪 Test RMSE:        {row['test_rmse']:.2f}")



📌 Dataset: One-hot completo
🔧 Best Val Params:  {'n_trees': 200, 'max_depth': 50, 'min_samples_split': 5, 'max_features': 20}
📉 Val RMSE:         6946.66
🏆 Best Test Params: {'n_trees': 200, 'max_depth': 50, 'min_samples_split': 2, 'max_features': 20}
🧪 Test RMSE:        4793.33

📌 Dataset: One-hot sin agrupamiento
🔧 Best Val Params:  {'n_trees': 100, 'max_depth': None, 'min_samples_split': 2, 'max_features': 20}
📉 Val RMSE:         7052.94
🏆 Best Test Params: {'n_trees': 200, 'max_depth': None, 'min_samples_split': 2, 'max_features': 20}
🧪 Test RMSE:        4839.08

📌 Dataset: Sin marcas con agrupamiento
🔧 Best Val Params:  {'n_trees': 20, 'max_depth': 50, 'min_samples_split': 10, 'max_features': 5}
📉 Val RMSE:         9294.81
🏆 Best Test Params: {'n_trees': 20, 'max_depth': 50, 'min_samples_split': 10, 'max_features': 5}
🧪 Test RMSE:        8357.68

📌 Dataset: Sin marcas sin agrupamiento
🔧 Best Val Params:  {'n_trees': 200, 'max_depth': 20, 'min_samples_split': 10, 'max_features': 2

In [25]:
# Dataset 2: agregando features numéricas derivadas
dp_numeric_feats = DataProcessor(df_raw, config={
    "add_precio_por_km": True,
    "add_antiguedad_squared": True,
    "add_cilindrada_times_km": True,
    "add_frecuencia_features": False
})
df_numeric_feats = dp_numeric_feats.preprocess()
df_numeric_feats.to_csv("dataset/processed/dataset_numeric_feats.csv", index=False)


# Dataset 3: agregando también frecuencia de marcas
dp_all_feats = DataProcessor(df_raw, config={
    "add_precio_por_km": True,
    "add_antiguedad_squared": True,
    "add_cilindrada_times_km": True,
    "add_frecuencia_features": True
})
df_all_feats = dp_all_feats.preprocess()
df_all_feats.to_csv("dataset/processed/dataset_all_feats.csv", index=False)

In [30]:
exploratory_analysis(df_all_feats, target_col="Precio_usd")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17781 entries, 0 to 17780
Columns: 194 entries, Puertas to frecuencia_modelo
dtypes: bool(182), float64(10), int64(2)
memory usage: 4.7 MB
None

--- Resumen Estadístico ---
            Puertas    Kilómetros     Precio_usd     Antigüedad    Cilindrada  \
count  17781.000000  1.778100e+04   17781.000000   17781.000000  17781.000000   
mean      18.522243  7.518157e+05   27390.631641     -18.399753      1.817519   
std      903.547364  1.064227e+06   19624.074249    3259.772123      0.625942   
min        2.000000  0.000000e+00     100.399912 -434669.000000      1.000000   
25%        5.000000  0.000000e+00   16021.716754       1.000000      1.500000   
50%        5.000000  6.800000e+05   23454.769418       6.000000      1.600000   
75%        5.000000  1.160000e+06   32229.216374       9.000000      2.000000   
max    60252.000000  9.999999e+07  610000.000000      39.000000      6.400000   

       Transmisión_Manual  vendedor_particular 

In [ ]:
resultados = []

datasets_explorations = [
    ("One-hot completo", df_OH),
    ("Dataset con features numéricas", df_numeric_feats),
    ("Dataset con features completas", df_all_feats)
]

for nombre, dataset in datasets_explorations:
    print(f"\n📊 Evaluando dataset: {nombre}")
    res = tune_and_test_rf(dataset, target_col="Precio_usd", param_grid=param_grid, top_n=3)
    print(res)
    resultados.append({
    "dataset": nombre,
    "best_params_val": res["best_params_val"],
    "val_rmse": res["val_rmse"],
    "best_params_test": res["best_params_test"],
    "test_rmse": res["test_rmse"]
})






📊 Evaluando dataset: One-hot completo
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 2, 'max_features': 5} --> RMSE promedio: 16565.2956
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 2, 'max_features': 20} --> RMSE promedio: 13058.9772
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 2, 'max_features': 'sqrt'} --> RMSE promedio: 13976.8248
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 2, 'max_features': 'log2'} --> RMSE promedio: 15733.0827
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 5, 'max_features': 5} --> RMSE promedio: 16576.6969
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 5, 'max_features': 20} --> RMSE promedio: 13211.9844
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 5, 'max_features': 'sqrt'} --> RMSE promedio: 14095.3699
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 5, 'max_features': 'log2'} --> RMSE promedio: 15847.7209
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 10, 'max_features': 5} --> RMSE promedio: 1659

In [ ]:
# Ahora, para ver resultados ordenados por test_rmse, por ejemplo:
import pandas as pd

df_resultados = pd.DataFrame(resultados)
df_resultados = df_resultados.sort_values(by="test_rmse")
print("\n📈 Ranking de datasets por menor RMSE en test:")
print(df_resultados.to_string(index=False))

# NN

In [3]:
dp_OH = DataProcessor(df_raw)
df_OH = pd.DataFrame(dp_OH.preprocess())
df_OH.to_csv("dataset/processed/marcas_OH.csv", index=False)
df_OH_R = df_OH.sample(frac=1, random_state=68).reset_index(drop=True)
df_OH_X = df_OH_R.drop(columns=["Precio_usd"])
df_OH_y = df_OH_R["Precio_usd"]
col = df_OH_X.columns.tolist()

Filas iniciales: 18254 → después del filtrado: 12458


In [4]:
from src.train_val_models import*
from models.nn import NeuralNetwork
from models.nn import cross_validate_nn

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2025-07-02 10:26:54.122107: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-02 10:26:54.122612: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-02 10:26:54.125374: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-02 10:26:54.133581: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cu

In [9]:
param_grid = {
    'hidden_layers': [
        [32],
        [64, 32],
        [128, 64, 32],
        [256, 128, 128]
        
    ],
    'optimizer': ['adam', 'rmsprop'],
    'learning_rate': [0.01, 0.001]
}

results = cross_validate_nn(df_OH_X, df_OH_y, param_grid, input_dim=df_OH_X.shape[1], epochs=200)

Evaluando: layers=[32], optimizer=adam, lr=0.01
Epoch 1/200
253/253 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 693339776.0000 - mae: 23710.1660 - mse: 693339776.0000 - val_loss: 634738368.0000 - val_mae: 22729.7207 - val_mse: 634738368.0000
Epoch 2/200
253/253 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 589179072.0000 - mae: 21743.9434 - mse: 589179072.0000 - val_loss: 457123424.0000 - val_mae: 18735.2324 - val_mse: 457123424.0000
Epoch 3/200
253/253 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 401116992.0000 - mae: 17235.1152 - mse: 401116992.0000 - val_loss: 253293088.0000 - val_mae: 12746.3516 - val_mse: 253293088.0000
Epoch 4/200
253/253 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 216355616.0000 - mae: 11259.0752 - mse: 216355616.0000 - val_loss: 123272072.0000 - val_mae: 7739.1729 - val_mse: 123272072.0000
Epoch 5/200
253/253 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 127661256.0000 - mae: 7177.8550 - mse: 127661256.0000 - val_loss: 70776344.0000 - val_mae: 5768.9922 - val_mse: 70776344.0000
Ep

In [10]:
results.sort(key=lambda x: x['avg_val_mse'])
for res in results[:5]:
    print(f"Layers: {res['hidden_layers']}, Optimizer: {res['optimizer']}, LR: {res['learning_rate']}, MSE: {res['avg_val_mse']:.4f}, RMSE: {np.sqrt(res['avg_val_mse']):.4f}")
    

Layers: [256, 128, 128], Optimizer: rmsprop, LR: 0.001, MSE: 17517887.0002, RMSE: 4185.4375
Layers: [128, 64, 32], Optimizer: rmsprop, LR: 0.01, MSE: 17560481.6543, RMSE: 4190.5228
Layers: [128, 64, 32], Optimizer: rmsprop, LR: 0.001, MSE: 17582705.5743, RMSE: 4193.1737
Layers: [256, 128, 128], Optimizer: adam, LR: 0.01, MSE: 17619662.2009, RMSE: 4197.5781
Layers: [256, 128, 128], Optimizer: rmsprop, LR: 0.01, MSE: 17647976.1099, RMSE: 4200.9494


In [11]:
#mejor modeelo
param_grid = {
    'hidden_layers': [
        [128, 64, 32]
        
    ],
    'optimizer': ['rmsprop'],
    'learning_rate': [0.001]
}

results = cross_validate_nn(df_OH_X, df_OH_y, param_grid, input_dim=df_OH_X.shape[1], epochs=100, k=5)
res = results[0]
print(f"Layers: {res['hidden_layers']}, Optimizer: {res['optimizer']}, LR: {res['learning_rate']}, MSE: {res['avg_val_mse']:.4f}, RMSE: {np.sqrt(res['avg_val_mse']):.4f}")

Evaluando: layers=[128, 64, 32], optimizer=rmsprop, lr=0.001
Epoch 1/100
304/304 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 664616320.0000 - mae: 23267.0137 - mse: 664616320.0000 - val_loss: 289189952.0000 - val_mae: 13344.8525 - val_mse: 289189952.0000
Epoch 2/100
304/304 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 143322448.0000 - mae: 8301.4326 - mse: 143322448.0000 - val_loss: 41832896.0000 - val_mae: 4292.9561 - val_mse: 41832896.0000
Epoch 3/100
304/304 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 64998872.0000 - mae: 3984.5278 - mse: 64998872.0000 - val_loss: 26618664.0000 - val_mae: 3260.1128 - val_mse: 26618664.0000
Epoch 4/100
304/304 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 28536750.0000 - mae: 3119.4199 - mse: 28536750.0000 - val_loss: 21977986.0000 - val_mae: 2874.1675 - val_mse: 21977986.0000
Epoch 5/100
304/304 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 26342426.0000 - mae: 2942.6509 - mse: 26342426.0000 - val_loss: 20125170.0000 - val_mae: 2775.4866 - val_mse: 20125170.0000
Epoch 

In [6]:

# preparo el dataset para entrenar el mejor modelo


train_df, temp_df = train_test_split(df_OH, test_size=0.4, random_state=12)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=12)

train_idx = train_df.index
val_idx = val_df.index
test_idx = test_df.index

X_train_df = df_OH_X.loc[train_idx]
y_train_df = df_OH_y.loc[train_idx]
X_train = df_OH_X.loc[train_idx].to_numpy()
y_train = df_OH_y.loc[train_idx].to_numpy()


X_val = df_OH_X.loc[val_idx].to_numpy()
y_val = df_OH_y.loc[val_idx].to_numpy()

X_test = df_OH_X.loc[test_idx].to_numpy()
y_test = df_OH_y.loc[test_idx].to_numpy()

#normalizo
dp = DataProcessor(df=None)
X_train = dp.normalize(X_train)
X_val = dp.normalize_new_data(X_val)
X_test = dp.normalize_new_data(X_test)

In [7]:
# mejor modelo listo para hacer las predicioes
nn = NeuralNetwork(input_dim=X_train.shape[1],
                    hidden_layers=[128, 64, 32],
                    optimizer_name='rmsprop',
                    learning_rate=0.001)
nn.fit(X_train, y_train,
    epochs=1000,
    batch_size=32,
    validation_data=(X_val, y_val))


Epoch 1/1000
228/228 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 691757056.0000 - mae: 23861.3457 - mse: 691757056.0000 - val_loss: 531150624.0000 - val_mae: 20367.6113 - val_mse: 531150624.0000
Epoch 2/1000
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 409024352.0000 - mae: 16318.1162 - mse: 409024352.0000 - val_loss: 79362912.0000 - val_mae: 6003.6455 - val_mse: 79362912.0000
Epoch 3/1000
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 93368848.0000 - mae: 5603.3408 - mse: 93368848.0000 - val_loss: 40040456.0000 - val_mae: 4146.3838 - val_mse: 40040456.0000
Epoch 4/1000
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 45832124.0000 - mae: 4018.3330 - mse: 45832124.0000 - val_loss: 29608808.0000 - val_mae: 3412.1047 - val_mse: 29608808.0000
Epoch 5/1000
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 30808626.0000 - mae: 3325.6423 - mse: 30808626.0000 - val_loss: 24692898.0000 - val_mae: 3034.6550 - val_mse: 24692898.0000
Epoch 6/1000
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss:

In [8]:
from sklearn.metrics import mean_squared_error
y_pred_val = nn.predict(X_val)
mse = mean_squared_error(y_val, y_pred_val)
print(f"Validation MSE: {mse:.4f}, RMSE: {np.sqrt(mse):.4f}")
y_pred_test = nn.predict(X_test)
mse = mean_squared_error(y_test, y_pred_test)
print(f"Test MSE: {mse:.4f}, RMSE: {np.sqrt(mse):.4f}")

76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
Validation MSE: 11294329.8462, RMSE: 3360.7038
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 619us/step
Test MSE: 11117287.7941, RMSE: 3334.2597


# Codigo extra para deteccion de outliers


In [12]:
#busco los valores que dan mal

train = df_OH_X.copy().to_numpy()
dp = DataProcessor(df=None)
train = dp.normalize(train)
train_y = df_OH_y.to_numpy()
nn = NeuralNetwork(input_dim=train.shape[1],
                    hidden_layers=[128, 64, 32],
                    optimizer_name='rmsprop',
                    learning_rate=0.001)
nn.fit(train, train_y,
    epochs=100,
    batch_size=32,
    validation_data=(train, train_y))


Epoch 1/100
556/556 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 958413696.0000 - mae: 24072.3770 - mse: 958413696.0000 - val_loss: 272972608.0000 - val_mae: 8668.3359 - val_mse: 272972608.0000
Epoch 2/100
556/556 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 291198176.0000 - mae: 7872.5996 - mse: 291198176.0000 - val_loss: 160851872.0000 - val_mae: 6261.0308 - val_mse: 160851872.0000
Epoch 3/100
556/556 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 136814304.0000 - mae: 6108.8706 - mse: 136814304.0000 - val_loss: 134897328.0000 - val_mae: 5702.1411 - val_mse: 134897328.0000
Epoch 4/100
556/556 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 127512832.0000 - mae: 5434.2012 - mse: 127512832.0000 - val_loss: 121539144.0000 - val_mae: 5292.3418 - val_mse: 121539144.0000
Epoch 5/100
556/556 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 146196096.0000 - mae: 5068.3096 - mse: 146196096.0000 - val_loss: 120702688.0000 - val_mae: 4849.8008 - val_mse: 120702688.0000
Epoch 6/100
556/556 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step -

In [ ]:
from sklearn.metrics import mean_squared_error
y_pred = nn.predict(train)
mse = mean_squared_error(train_y, y_pred)
print(f"Validation MSE: {mse:.4f}, RMSE: {np.sqrt(mse):.4f}")
squared_errors = (train_y - y_pred.ravel()) ** 2


# Add the squared errors as a new column to the DataFrame
train_with_errors = df_OH_X.copy()
train_with_errors['squared_error'] = squared_errors
train_with_errors = train_with_errors.sort_values(by='squared_error', ascending=False)
def collapse_one_hot_columns(df, prefix):
    cols = [col for col in df.columns if col.startswith(prefix + '_')]
    if not cols:
        return df
    df[prefix] = df[cols].idxmax(axis=1).str[len(prefix)+1:]
    # Drop original one-hot columns
    df = df.drop(columns=cols)
    return df


for prefix in ['Marca', 'Modelo', "combustible"]:
    train_with_errors = collapse_one_hot_columns(train_with_errors, prefix)


train_with_errors.to_csv("dataset/processed/train_with_errors.csv", index=False)

556/556 ━━━━━━━━━━━━━━━━━━━━ 0s 512us/step
Validation MSE: 120702675.1226, RMSE: 10986.4769
